### Part I: run the code in Springboard database user interface.URL: https://sql.springboard.com/

<span style='color:Blue'> Question 1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.</span>

Q1= SELECT `name` FROM `Facilities` WHERE `membercost` != 0

<span style='color:Blue'> Question 2: How many facilities do not charge a fee to members? </span>

Q2=SELECT COUNT(`name`) FROM `Facilities` WHERE `membercost` = 0

<span style='color:Blue'> Question 3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.</span>

Q3=SELECT `facid`, `name`, `membercost`, `monthlymaintenance` FROM `Facilities` WHERE `membercost` < (0.2* `monthlymaintenance`)

<span style='color:Blue'> Question 4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. </span>

Q4=SELECT * FROM `Facilities` WHERE `facid` IN (1,5)

<span style='color:Blue'> Question 5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. </span>

Q5=SELECT `facid`, `name`, CASE WHEN `monthlymaintenance` > 100 THEN 'expensive' ELSE 'cheap' END AS `monthly_maintenance` FROM `Facilities`

<span style='color:Blue'> Question 6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. </span>

Q6 = SELECT `memid`, `surname`, `firstname` FROM `Members` WHERE `memid` != 0 AND `joindate`=(SELECT MAX(`joindate`) FROM `Members`)

<span style='color:Blue'> Question 7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.  </span>

Q7= SELECT DISTINCT `facility_name`, concat(`surname`, ' ', `firstname`) AS `member_name` FROM (SELECT `memid`, `Facilities`.`name` AS `facility_name`, `Bookings`.`facid` FROM `Bookings` LEFT JOIN `Facilities` ON `Bookings`.`facid` = `Facilities`.`facid`) AS `BF` LEFT JOIN `Members` ON `BF`.`memid` = `Members`.`memid` WHERE `BF`.`facid` IN (SELECT `facid` FROM `Facilities` WHERE `name` LIKE 'tennis court%') AND `surname` != 'guest'  ORDER BY member_name

<span style='color:Blue'> Question 8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.  </span>

Q8= SELECT  `Facilities`.`name`, CONCAT(`Members`.`surname`, ' ', `Members`.`firstname`), CASE WHEN `Bookings`.`memid` = 0 THEN `slots`*`guestcost`*2 ELSE `slots`*`membercost`*2 END AS `slot_cost` FROM `Bookings` LEFT JOIN `Facilities` ON `Bookings`.`facid` = `Facilities`.`facid` LEFT JOIN `Members` ON `Bookings`.`memid` = `Members`.`memid` WHERE ((`Bookings`.`starttime` LIKE '2012-09-14%') AND (`Bookings`.`memid`!= 0) AND (`slots`*`membercost`*2 >30)) OR ((`Bookings`.`starttime` LIKE '2012-09-14%') AND (`Bookings`.`memid`= 0) AND (`slots`*`guestcost`*2 >30)) ORDER BY `slot_cost` DESC

<span style='color:Blue'> Question 9: This time, produce the same result as in Q8, but using a subquery. </span>

Q9= SELECT  `BF`.`name`, CONCAT(`Members`.`surname`, ' ', `Members`.`firstname`), CASE WHEN `BF`.`memid` = 0 THEN `slots`*`guestcost`*2 ELSE `slots`*`membercost`*2 END AS `slot_cost` FROM
(SELECT `Facilities`.`name`, `Facilities`.`membercost`, `Facilities`.`guestcost`, `Bookings`.`memid`, `Bookings`.`starttime`, `Bookings`.`slots` FROM `Bookings` LEFT JOIN `Facilities` ON `Facilities`.`facid` = `Bookings`.`facid`) AS `BF` LEFT JOIN `Members` ON `BF`.`memid` = `Members`.`memid` WHERE ((`BF`.`starttime` LIKE '2012-09-14%') AND (`BF`.`memid`!= 0) AND (`slots`*`membercost`*2 >30)) OR ((`BF`.`starttime` LIKE '2012-09-14%') AND (`BF`.`memid`= 0) AND (`slots`*`guestcost`*2 >30)) ORDER BY `slot_cost` DESC

### Part II: Use local DB file to complete Q10-13

Some functions work on the website, https://sql.springboard.com, but not in sqlite. For example, concat() is not supported in sqlite. The same thing happened for month(). If you test the code using the website, please refer to the markdown cells Q1-Q13. If you run in Jupyter notebook, please use the code in the normal cells.


In [7]:
from sqlalchemy import create_engine
import pandas as pd
import sqlite3

In [8]:
engine = create_engine('sqlite:///C:/Users/gln_d/Desktop/Springboard/mini_project/mini_project_Spring_Board/sqlite_db_pythonsqlite.db')

Q10 = SELECT `name`, SUM(`slot_revenue`) AS `facility_revenue` FROM (SELECT `name`, CASE WHEN `memid` = 0 THEN `slots`*`guestcost`*2 ELSE `slots`*`membercost`*2 END AS `slot_revenue`  FROM (SELECT `bookid`, `memid`, `starttime`, `slots`, `name`, `membercost`, `guestcost` FROM `Bookings` LEFT JOIN `Facilities` ON `Bookings`.`facid`=`Facilities`.`facid`) AS `BF`) AS `FF` GROUP BY `name` ORDER BY `name`

<span style='color:Blue'> Question 10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! </span>

In [3]:
Q10 = pd.read_sql_query("SELECT name, SUM(slot_revenue) AS facility_revenue FROM (SELECT name, CASE WHEN memid = 0 THEN slots*guestcost*2 ELSE slots*membercost*2 END AS slot_revenue FROM (SELECT bookid, memid, starttime, slots, name, membercost, guestcost FROM Bookings LEFT JOIN Facilities ON Bookings.facid=Facilities.facid) AS BF) AS FF GROUP BY name ORDER BY name", engine)
print(Q10)

              name  facility_revenue
0  Badminton Court            3813.0
1   Massage Room 1          100703.2
2   Massage Room 2           28909.2
3       Pool Table             540.0
4    Snooker Table             480.0
5     Squash Court           26936.0
6     Table Tennis             360.0
7   Tennis Court 1           27720.0
8   Tennis Court 2           28620.0


<span style='color:Blue'> Question 11: Produce a report of members and who recommended them in alphabetic surname,firstname order </span>

Q11=SELECT CONCAT(`member_surname`, ' ', `member_firstname`) AS `Member_name`, CONCAT(`recommender_surname`, ' ', `recommender_firstname`) AS `recommender_name` FROM (SELECT `a`.`memid` AS `member_id`, `a`.`surname` AS `member_surname`, `a`.`firstname` AS `member_firstname`,`b`.`memid` AS `recommend_id`,`b`.`surname` AS `recommender_surname`, `b`.`firstname` AS `recommender_firstname` FROM `Members` `a`, `Members` `b` WHERE `a`.`memid`= `b`.`recommendedby`)AS `Members_sj` WHERE `member_id` != 0 ORDER BY `Member_name`

In [12]:
Q11 = pd.read_sql_query("SELECT member_surname || ' ' || member_firstname AS Member_name, recommender_surname || ' ' || recommender_firstname AS recommender_name FROM (SELECT a.memid AS member_id, a.surname AS member_surname, a.firstname AS member_firstname,b.memid AS recommend_id,b.surname AS recommender_surname, b.firstname AS recommender_firstname FROM Members a, Members b WHERE a.memid= b.recommendedby) AS Members_sj WHERE member_id != 0 ORDER BY Member_name", engine)
print(Q11)

          Member_name         recommender_name
0      Bader Florence         Sarwin Ramnaresh
1       Baker Timothy              Coplin Joan
2      Butters Gerald          Genting Matthew
3      Farrell Jemima            Baker Timothy
4      Farrell Jemima             Pinker David
5     Genting Matthew         Rumney Henrietta
6         Jones David            Jones Douglas
7     Joplette Janice               Dare Nancy
8     Joplette Janice              Jones David
9   Purview Millicent                Hunt John
10         Rownam Tim               Boothe Tim
11       Smith Darren          Joplette Janice
12       Smith Darren           Butters Gerald
13       Smith Darren             Owen Charles
14       Smith Darren               Smith Jack
15       Smith Darren           Mackenzie Anna
16        Smith Tracy  Worthington-Smyth Henry
17        Smith Tracy        Purview Millicent
18        Smith Tracy            Crumpet Erica
19    Stibbons Ponder               Baker Anne
20    Stibbon

Q12=SELECT `Bookings`.`facid`, `Facilities`.`name`, COUNT(`bookid`) FROM `Bookings` LEFT JOIN `Facilities` ON `Bookings`.`facid`=`Facilities`.`facid` WHERE `Bookings`.`memid` != 0 GROUP BY `name`

<span style='color:Blue'> Question 12: Find the facilities with their usage by member, but not guests </span>

In [15]:
Q12 = pd.read_sql_query("SELECT Bookings.facid, Facilities.name, COUNT(bookid) FROM Bookings LEFT JOIN Facilities ON Bookings.facid=Facilities.facid WHERE Bookings.memid != 0 GROUP BY name", engine)
print(Q12)

   facid             name  COUNT(bookid)
0      2  Badminton Court            344
1      4   Massage Room 1            421
2      5   Massage Room 2             27
3      8       Pool Table            783
4      7    Snooker Table            421
5      6     Squash Court            195
6      3     Table Tennis            385
7      0   Tennis Court 1            308
8      1   Tennis Court 2            276


<span style='color:Blue'> Question 13: Find the facilities usage by month, but not guests </span>

Q13 = SELECT `facid`, `month`, SUM(`slots`) FROM (SELECT `facid`, MONTH(`starttime`) AS `month`, `slots`, `memid` FROM `Bookings`) AS `Booking_by_month` WHERE `memid` != 0 GROUP By `month`, `facid`

In [17]:
Q13 = pd.read_sql_query(" SELECT facid, month, SUM(slots) FROM (SELECT facid, strftime('%m', starttime) AS month, slots, memid FROM Bookings) AS Booking_by_month WHERE memid != 0 GROUP By month, facid", engine)
print(Q13)

    facid month  SUM(slots)
0       0    07         201
1       1    07         123
2       2    07         165
3       3    07          98
4       4    07         166
5       5    07           8
6       6    07          50
7       7    07         140
8       8    07         110
9       0    08         339
10      1    08         345
11      2    08         414
12      3    08         296
13      4    08         316
14      5    08          18
15      6    08         184
16      7    08         316
17      8    08         303
18      0    09         417
19      1    09         414
20      2    09         507
21      3    09         400
22      4    09         402
23      5    09          28
24      6    09         184
25      7    09         404
26      8    09         443
